In [1]:
from dash import dcc, html
from dash.dependencies import Input, Output
from urllib.parse import parse_qs

from procesos.comerciales import *
from procesos.ficheros import leer_comerciales


In [2]:
ventas = None
ventas = leer_comerciales("c:\datos\[OPORT021]_Oportunidades_con_multiples_filtros_RAW.xls", ventas)
print('Primera lectura')
ventas = preprocesar(ventas)

Estamos leyendo fichero
Primera lectura
procesamos DF


In [3]:
ventas.head(10)

,Comercial,Cliente,Oportunidad,Estado,Motivos,ObservacionesMotivo,Avance,Probabilidad,Importeprevisto,Duraciónmeses,...,CódigoLead,Tiposdeservicio,Solución,Lead,GeneraPipeline,País,Importepond.LoB,Importepond.LoBenPipe,Fecha,Anualidad
0,Pilar Gomez Navarro,[KIAM] KIA IBERIA S.L.U,[5013] Implantación JIRA,Cerrada y ganada,Solvencia técnica,Cliente muy contento con nuestros conocimiento...,2 - Elaboración de Propuesta,0.40,9640.00,1.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201806,2018
1,Pilar Gomez Navarro,[KIAM] KIA IBERIA S.L.U,[5014] Firma Electrónica en Tablet,Cerrada y ganada,Solvencia técnica,Solvencia técnica,3 - Presentada/Negociación,0.30,22500.00,3.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201806,2018
2,José María del Río Cebrián,[CTBN] Cáritas Diocesana de Barcelona,[5004] Gestión de Subvenciones con Jira,Cerrada y ganada,"Somos los únicos en presentarnos, Solvencia té...",,3 - Presentada/Negociación,0.25,9920.00,2.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,9920,2480,201807,2018
3,Laura Casorran Garralaga,[IBVI] IBERCAJA VIDA COMPAÑÍA DE SEGUROS Y REA...,[5001] Análisis renovación del entorno informá...,Cerrada y perdida,Otros,"Oportunidad para presentación servicios, a Ibe...",0 - Prospect,0.05,0.00,1.0,...,,Precio Cerrado,Sin solución,No,No,atEspaña,0,0,201807,2018
4,Miguel Angel Valiente Alvarez,[ECIN] El Corte Inglés S.A.,[5128] Servicio de Arquitectura de API Management,Cerrada y no presentada,Mantenimiento,La dirección decidió cerrar la oportunidad el ...,1 - Cualificación,0.60,140000.00,7.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,140000,84000,201807,2018
5,Luis Ignacio García Valtuille,[ALSA] alsa,[5003] Bolsa Horas Atlassian,Cerrada y ganada,Solvencia técnica,Ampliacion de las labores de proyecto y horas ...,3 - Presentada/Negociación,0.50,10000.00,12.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201807,2018
6,Bernardo Blanco Zapata,[CION] CIONE,[5001] Implantación e integración de Magnolia ...,Cerrada y ganada,"Solvencia técnica, Currículum del equipo prese...","En competencia con Deloitte y Vass, el equipo...",3 - Presentada/Negociación,0.30,182365.00,7.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201807,2018
7,Oscar Muñoz Cabañas,"[AEGO] AEGON ADMINISTRACION Y SERVICIOS, A.I.E.",[5048] Renovacion Atlassian SSO,Cerrada y ganada,Solvencia técnica,Renovacion atlassian SSO,3 - Presentada/Negociación,0.30,834.78,12.0,...,,Venta Software,Sin solución,No,Sí,atEspaña,,,201807,2018
8,Oscar Muñoz Cabañas,"[OFCU] UNIVERSITAS XXI, Soluciones y Tecnologí...",[5106] Evolucion empaquetado,Cerrada y ganada,"Somos los únicos en presentarnos, Solvencia té...",Conocemos muy bien el sistema de empaquetado p...,3 - Presentada/Negociación,0.30,11520.00,2.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201807,2018
9,Bernardo Blanco Zapata,[BUPA] BUPA Investments Overseas Limited Sucursal,[5012] Mi Salud Bucodental_Dental Anamnesis,Cerrada y perdida,Precio algo mayor que la oferta ganadora,Se la otorgan a un proveedor que ya ha trabaja...,2 - Elaboración de Propuesta,0.30,10000.00,1.0,...,,Precio Cerrado,Sin solución,No,Sí,atEspaña,,,201807,2018


In [11]:
nombre_comercial = "Andreu Blancas Sanchez"
fechas_df = comercial_fechas(ventas, nombre_comercial)
fechas_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, 0 to 245
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   index                   246 non-null    int64         
 1   Comercial               246 non-null    object        
 2   Cliente                 246 non-null    object        
 3   Oportunidad             246 non-null    object        
 4   Estado                  246 non-null    object        
 5   Motivos                 246 non-null    object        
 6   ObservacionesMotivo     246 non-null    object        
 7   Avance                  246 non-null    object        
 8   Probabilidad            246 non-null    float64       
 9   Importeprevisto         246 non-null    float64       
 10  Duraciónmeses           246 non-null    float64       
 11  Fechacreación           246 non-null    datetime64[ns]
 12  Fechamodificación       246 non-null    datetime64[ns]


In [12]:
def tipo_servicio_contrato(ventas, nombre):
    # agrupamos, para un comercial los tipos de servicio
    ventas_cerradas = pd.DataFrame(ventas[(ventas['Comercial'] == nombre) & (ventas['Estado'] == 'Cerrada y ganada')])
    nuevo_df = ventas_cerradas.groupby(['Tiposdeservicio', 'Anualidad'])['Importeprevisto'].sum().reset_index()
    nuevo_df.reset_index(inplace=True)
    nuevo_df.columns.name = None
    nuevo_df = nuevo_df.fillna(0)

    return nuevo_df



tipo_servicio = tipo_servicio_contrato(ventas, nombre_comercial)
tipo_servicio.head(10)

,index,Tiposdeservicio,Anualidad,Importeprevisto
0,0,Bolsa de Horas,2020,26600.00
1,1,Bolsa de Horas,2021,36450.49
2,2,Bolsa de Horas,2022,38500.00
3,3,Bolsa de Horas,2023,26800.00
4,4,Formación,2020,18500.00
5,5,Formación,2021,2352.00
6,6,Gestionado,2019,5000.00
7,7,Gestionado,2020,37600.00
8,8,Gestionado,2021,92187.00
9,9,Gestionado,2022,0.00


In [13]:
import plotly.express as px


grouped = tipo_servicio.groupby(['Tiposdeservicio', 'Anualidad'])['Importeprevisto'].sum().reset_index()


fig = px.bar(tipo_servicio, x="Anualidad", y="Importeprevisto", color="Tiposdeservicio", title="Ventas por tipos de servicio", text="Tiposdeservicio")


fig.show()